In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `C:\Users\tge13\Documents\optimal_control_julia\lec_14\Project.toml`


In [2]:
using LinearAlgebra
using ForwardDiff

## Skew-Symmetrix Mat

In [3]:
function hat(v)
    return [
        0 -v[3] v[2];
        v[3] 0 -v[1];
        -v[2] v[1] 0
    ]
end

hat (generic function with 1 method)

## Quaternion Multiplication Mat

In [ ]:
function L(q)
    s = q[1]
    v = q[2:4]
    return [
        s -v';
        v s*I+hat(v)
    ]
end

function R(q)
    s = q[1]
    v = q[2:4]
    return [
        s -v';
        v s*I-hat(v)
    ]
end

## Helper Mat

In [ ]:
T = Diagonal([1, -1, -1, -1])
H = [zeros(3)'; I(3)]

function G(q)
    return L(q)*H
end

function Q(q)
    return H'*(R(q)'*L(q))*H
end

In [12]:
q_true = randn(4)
q_true = q_true/norm(q_true)

vN = randn(3, 10)
for k in 1:10
    vN[:,k] .= vN[:,k]/norm(vN[:,k])
end

In [13]:
Q_true = Q(q_true)
vB = Q_true'*vN

3×10 Matrix{Float64}:
 -0.910386   0.345562  -0.782959  …   0.017407  0.498702   -0.665686
  0.37988    0.330592   0.366995     -0.398513  0.0625587   0.695516
  0.163978  -0.878234  -0.502284     -0.916997  0.864513    0.270406

In [27]:
function residual(q)
    # caution! residual return is not a matrix but a vector
    r = vN - Q(q)*vB
    return r[:]
end

residual (generic function with 1 method)

In [41]:
#Random initial guess
q = randn(4)
q = q/norm(q)

4-element Vector{Float64}:
 -0.1461099099841532
  0.291198557543388
  0.9009763720212778
 -0.28652551605127335

## Gradient Gauss-Newton

In [42]:
ϕ = ones(3)
iter = 0
# @show maximum(abs.(ϕ))

while maximum(abs.(ϕ)) > 1e-8
    r = residual(q)
    dr = ForwardDiff.jacobian(residual, q)
    ∇r = dr*G(q)
    ϕ = -(∇r'*∇r)\(∇r'*r)
    q = L(q)*[ sqrt(1-ϕ'*ϕ); ϕ]
    iter += 1
end

In [43]:
iter

8

In [44]:
q - q_true

4-element Vector{Float64}:
 -1.1102230246251565e-16
  0.0
  0.0
  5.551115123125783e-17

In [45]:
q + q_true

4-element Vector{Float64}:
  1.7430418567535715
 -0.3787926392115736
  0.5833684596292392
 -0.6913772214891098